# Chapter 3_텐서플로우 기본

### simple model
<img src="./img/simple.jpeg" width=50%>

In [10]:
import tensorflow as tf

In [11]:
a = tf.constant(5, name="imput_a")
b = tf.constant(3, name="imput_b")

In [12]:
c = tf.multiply(a,b, name="mul_c")
d = tf.add(a,b, name="add_d")

In [13]:
e = tf.add(c,d, name="add_e")

In [16]:
sess = tf.Session()
sess.run(e)

23

### 개선사항
- 입력은 tf.constant 노드 대신에 __플레이스홀더를 사용.__
- 두개의 이산 스칼라(discrete scalar) 입력 대신에 __가변 길이의 하나의 벡터 사용.__
- 그래프를 사용하는 동안의 시간에 따른 __모든 출력값을 누적.__
- 그래프를 __namescope__로 깔끔하게 분할.
- 각각의 실행 후에 그래프의 출력, 모든 출력 결과의 누적, 모든 출력 결과의 평균을 __텐서보드에서 사용하기 위해서 디스크에 저장.__

### better model
<img src="./img/better.jpeg" width=50%>

In [19]:
import numpy as np

In [22]:
graph = tf.Graph()
with graph.as_default():
    
    ### 속성상 전역변수들인 것들 
    with tf.name_scope("variables"): 
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step") 
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="global_step")
        # trainable=False : 변수들이 수작업으로 셋팅될것이라고 명시
        
    ### 핵심 연산 부분    
    with tf.name_scope("transformation"): 
        
        # input layer
        with tf.name_scope("input"):
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
            
        # middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="plod_b")
            c = tf.reduce_sum(a, name="sum_c")
            
        # output layer
        with tf.name_scope("output"):
            output = tf.add(b,c, name="output")
            
            
    ### 트랜스포메이션 연산을 마친후에 두개의 변수를 업데이트하는 연산
    with tf.name_scope("update"):
        # 최근 output으로부터 total_output 변수 증가.
        update_total = total_output.assign_add(output)
        
        # 그래프가 run할때마다 global_step 변수 증가.
        increment_step = global_step.assign_add(1)
        
        
    ### 텐서보드 요약
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
        
        tf.summary.scalar(b'Output', )
            



23


In [ ]:
with tf.Session() as sess:
    a = tf.placeholder(tf.int32, shape=[None], name="my_input")

    b = tf.reduce_prod(a, name="plod_b")
    c = tf.reduce_sum(a, name="sum_c")

    d = tf.add(b,c, name="add_d")
    
    input_dict = {a: np.array([5,3], dtype=np.int32)}
    print(sess.run(d, feed_dict=input_dict))